In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

import sys
sys.path.append("..")

import src.support.data_upload_support as dus


In [6]:
enlaces_navegacion = ["https://minimalismbrand.com/collections/hogar",
                      "https://minimalismbrand.com/collections/mochilas",
                      "https://minimalismbrand.com/collections/camisetas-algodon-organico-nino-a",
                      "https://minimalismbrand.com/collections/ropa-minimalism-mujer",
                      "https://minimalismbrand.com/collections/ropa-minimalism"
                      ]

In [7]:
def sacar_productos_pagina(soup, diccionario_extract, diccionario):
    try:
        product_cards_list = soup.find("div",{"class":"product-list product-list--per-row-3 product-list--per-row-mob-2 product-list--per-row-mob-2 product-list--image-shape-shortest"}).findChildren("div",recursive=False)
    except:
        print()
        return diccionario
    for product_card in product_cards_list:
        for campo, valor_func in diccionario_extract.items():
            try:
                diccionario[campo].append(valor_func(product_card))
        
            except:
                diccionario[campo].append(np.nan)
    return diccionario

Diccionarios funciones extraccion

In [8]:
diccionario_extract = {
    "nombre": lambda card: re.sub(r"\s+"," ",card.find("h3",{"class":"product-block__title"}).text.strip()),
    "precio": lambda card: card.find("div",{"class":"product-price"}).findChildren("span",recursive=False)[0].text.strip(),
    "precio_original": lambda card: card.find("div",{"class":"product-price"}).findChildren("span",recursive=False)[1].text.strip(),
    "url": lambda card: "https://minimalismbrand.com"+ card.find("a",{"class":"product-link"})["href"]
}


In [9]:
diccionario = {
        "nombre": [],
        "precio": [],
        "precio_original": [],
        "url": []
}
for enlace in enlaces_navegacion:
    print("Enlace principal", enlace)

    # 
    response = requests.get(enlace)
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # enlaces en subcategoría
    enlace_subcategoria = soup.find("a",{"class":"image-overlay__image-link"})

    # SI ES PAGINA SUBCATEGORIAS
    if enlace_subcategoria:

        # ENCONTRAR ENLACES SUBCATEGORIAS, ENTRAR EN CADA PAGINA PRODUCTOS
        lista_enlace_pag_productos = ["https://minimalismbrand.com"+elemento["href"] for elemento in soup.find_all("a",{"class":"image-overlay__image-link"})]

        try:
            lista_nombres_productos = [elemento.find("span").text.strip() for elemento in soup.find_all("a",{"class":"image-overlay__image-link"}) if elemento and elemento.find("span")]
        except:
            # print("except")
            lista_nombres_productos = [elemento.find("span") for elemento in soup.find_all("a",{"class":"image-overlay__image-link"}) if elemento]

            
        # print(lista_nombres_productos)

        for enlace_pag_producto in lista_enlace_pag_productos: 
            # ENCONTRAR EN CADA URL PRODUCTO
            # print("Enlace_pag_producto", enlace_pag_producto)
            response = requests.get(enlace_pag_producto)
            subsoup = BeautifulSoup(response.text, 'html.parser')
            
            diccionario = sacar_productos_pagina(subsoup,diccionario_extract, diccionario)

    # SI ES PAGINA ENLACES A PRODUCTOS 
    else:
        diccionario = sacar_productos_pagina(soup,diccionario_extract, diccionario)

precios_minimalism = pd.DataFrame(diccionario)
precios_minimalism["precio"] = np.where(precios_minimalism["precio"]=="De", precios_minimalism["precio_original"], precios_minimalism["precio"])
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].fillna("-")
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].map(lambda x: x.replace("€",""))
precios_minimalism.shape

Enlace principal https://minimalismbrand.com/collections/hogar
Enlace principal https://minimalismbrand.com/collections/mochilas
Enlace principal https://minimalismbrand.com/collections/camisetas-algodon-organico-nino-a
Enlace principal https://minimalismbrand.com/collections/ropa-minimalism-mujer

Enlace principal https://minimalismbrand.com/collections/ropa-minimalism



(258, 4)

In [10]:
precios_minimalism["url2"] = precios_minimalism["url"].str.split("?",expand=True)[0]
precios_minimalism.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre           258 non-null    object
 1   precio           258 non-null    object
 2   precio_original  258 non-null    object
 3   url              258 non-null    object
 4   url2             258 non-null    object
dtypes: object(5)
memory usage: 10.2+ KB


In [11]:
precios_minimalism.drop_duplicates(subset="url2",inplace=True)
precios_minimalism.shape

(80, 5)

In [12]:
precios_minimalism

,nombre,precio,precio_original,url,url2
0,Juego de sábanas · Algodón orgánico - 105x200,70.00,-,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...
4,Toalla lavabo · Algodón orgánico - Blanco,9.00,-,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...
6,Toalla de ducha · Algodón orgánico - Blanco,17.00,-,https://minimalismbrand.com/products/toalla-du...,https://minimalismbrand.com/products/toalla-du...
8,Toallas baño pack mediano · Algodón orgánico,24.00,26.00,https://minimalismbrand.com/products/toallas-b...,https://minimalismbrand.com/products/toallas-b...
9,Gel sólido uso diario x3,20.00,22.50,https://minimalismbrand.com/products/gel-solid...,https://minimalismbrand.com/products/gel-solid...
...,...,...,...,...,...
180,Calzoncillos Slips Orgánicos X3,32.00,36.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
181,Calzoncillos Slips Orgánicos X5,50.00,60.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
182,Calzoncillos Slips Orgánicos X7,68.00,84.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
210,Pantalón orgánico Slim Hombre,70.00,-,https://minimalismbrand.com/products/pantalon-...,https://minimalismbrand.com/products/pantalon-...


In [13]:
precios_minimalism.to_csv("../datos/precios_minimalism.csv")

In [14]:
dus.upload_to_drive("../datos/precios_minimalism.csv", '1vfttgGDHa3-eCMEN79QAue6aI3Gc1vvp')

Archivo subido con ID: 1itrNHGjAzNEg3NdJGhsTEDIGt-FplLlU


## Pruebas beautiful soup pagina producto

In [15]:
url_pag_producto = "https://minimalismbrand.com/products/camiseta-algodon-organico?variant=34948534108317"
response = requests.get(url_pag_producto)

soup = BeautifulSoup(response.text, 'html.parser')

Conseguir vertical

In [16]:
" ".join(soup.find("div",{"class":"container pango-breadcrumb"}).text.strip().split()[2:])

'Camisetas algodón orgánico'

In [44]:
def extraer_info_producto(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text,"html.parser")

    diccionario_campos = {
        "score": soup.find("span",{"class":"stamped-badge-caption"})["data-rating"].replace(",",".") if soup.find("span",{"class":"stamped-badge-caption"}) else None,
        "reviews": soup.find("span",{"class":"stamped-badge-caption"})["data-reviews"] if soup.find("span",{"class":"stamped-badge-caption"}) else None,
        "vertical": " ".join(soup.find("div",{"class":"container pango-breadcrumb"}).text.strip().split()[2:]) if soup.find("div",{"class":"container pango-breadcrumb"}) else None
    }

    return [*diccionario_campos.values()]



In [45]:
[*extraer_info_producto("https://minimalismbrand.com/products/camiseta-algodon-organico?variant=34948534108317")]

['4.7', '405', 'Camisetas algodón orgánico']

In [56]:
df = pd.DataFrame(precios_minimalism[["url","url2"]][0:2])
df

,url,url2
0,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...
4,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...


In [57]:
df[["score","reviews","vertical"]] = df["url2"].apply(lambda x: extraer_info_producto(x)).to_list()

In [59]:
df

,url,url2,score,reviews,vertical
0,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...,3.7,3,Esenciales para tu hogar
4,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...,4.2,9,Esenciales para tu baño


Conseguir score

In [ ]:
score = soup.find("span",{"class":"stamped-badge-caption"})["data-rating"]


Conseguir reviews

In [ ]:
n_reviews = soup.find("span",{"class":"stamped-badge-caption"})["data-reviews"]